# 3.2章　LLM にツールを与える

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_\
API_KEY')

## 3.2.1 検索ツール

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

question = "株式会社Elithの住所を教えてください。最新の公式情報として\
公開されているものを教えてください。"

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke([HumanMessage(content=question)])

print(result.content)

In [ ]:
!pip install serpapi
!pip install google-search-results

In [ ]:
# load_toolsを利用するのに必要
!pip install langchain_community

In [ ]:
from langchain.agents import load_tools

model = ChatOpenAI(model="gpt-4o-mini")

#1 ツールをロード
tools = load_tools(["serpapi"], llm=model)

#2 LLMにツールを紐付け
model_with_tools = model.bind_tools(tools)

question = "株式会社Elithの住所を教えてください。最新の公式情報として\
公開されているものを教えてください。"

response = model_with_tools.invoke([HumanMessage(content=\
question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
tools = load_tools(["serpapi"], llm=model)
search_tool = tools[0]
search_tool.invoke(response.tool_calls[0]["args"])

## 3.2.2 プログラム実行ツール

In [ ]:
question = "以下をPythonで実行した場合の結果を教えてください。print\
(1873648+9285928+3759182+2398597)"

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke([HumanMessage(content=question)])

print(result.content)

In [ ]:
print(1873648+9285928+3759182+2398597)

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain_experimental.tools.python.tool import\
PythonREPLTool

model = ChatOpenAI(model="gpt-4o-mini")
tools = [PythonREPLTool()]
model_with_tools = model.bind_tools(tools)

question = "以下をPythonで実行した場合の結果を教えてください。print\
(1873648+9285928+3759182+2398597)"

response = model_with_tools.invoke([HumanMessage(content=\
question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
pythonrepltool = PythonREPLTool()
pythonrepltool.invoke(response.tool_calls[0]["args"])

## 3.2.3 ツールを自作する

In [ ]:
# おみくじ関数

import random
from datetime import datetime

def get_fortune(date_string):
    # 日付文字列を解析
    try:
        date = datetime.strptime(date_string, "%m月%d日")
    except ValueError:
        return "無効な日付形式です。'X月X日'の形式で入力してくださ\
い。"

    # 運勢のリスト
    fortunes = [
        "大吉", "中吉", "小吉", "吉", "末吉", "凶", "大凶"
    ]

    # 運勢の重み付け（大吉と大凶の確率を低くする）
    weights = [1, 3, 3, 4, 3, 2, 1]

    # 日付に基づいてシードを設定（同じ日付なら同じ運勢を返す）
    random.seed(date.month * 100 + date.day)

    # 運勢をランダムに選択
    fortune = random.choices(fortunes, weights=weights)[0]

    return f"{date_string}の運勢は【{fortune}】です。"

# 出力例
get_fortune("10月22日")

In [ ]:
from langchain.tools import BaseTool

#1 ツールの定義
class Get_fortune(BaseTool):
    name: str = 'Get_fortune'
    description: str = (
        "特定の日付の運勢を占う。インプットは  'date_string'です。\
'date_string' は、占いを行う日付で、mm月dd日 という形式です。「1月1日」\
のように入力し、「'1月1日'」のように余計な文字列を付けてはいけません。"
    )

    def _run(self, date_string) -> str:
        return get_fortune(date_string)


    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [ ]:
tools = [Get_fortune()]

model = ChatOpenAI(model="gpt-4o-mini")
model_with_tools = model.bind_tools(tools)

question = "10月22日の運勢を教えてください。"

response = model_with_tools.invoke([HumanMessage(content=\
question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
tool = Get_fortune()
tool.invoke(response.tool_calls[0])

In [ ]:
tools = [Get_fortune()]

model = ChatOpenAI(model="gpt-4o-mini")
model_with_tools = model.bind_tools(tools)

question = "今日の運勢を教えてください。"

response = model_with_tools.invoke([HumanMessage(content=\
question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from datetime import timedelta
from zoneinfo import ZoneInfo

# 日付取得関数

def get_date(date):
    date_now = datetime.now(ZoneInfo("Asia/Tokyo"))
    if ("今日" in date):
        date_delta = 0
    elif ("明日" in date):
        date_delta = 1
    elif ("明後日" in date):
        date_delta = 2
    else:
        return "サポートしていません"
    return (date_now + timedelta(days=date_delta)).strftime\
('%m月%d日')

# 出力例
print(get_date("今日"))

In [ ]:
class Get_date(BaseTool):
    name: str = 'Get_date'
    description: str = (
        "今日の日付を取得する。インプットは 'date'です。'date' は、日\
付を取得する対象の日で、'今日', '明日', '明後日' という3種類の文字列\
から指定します。「今日」のように入力し、「'今日'」のように余計な文字列を付\
けてはいけません。"
    )

    def _run(self, date) -> str:
        return get_date(date)

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [ ]:
tools = [Get_date()]

model = ChatOpenAI(model="gpt-4o-mini")
model_with_tools = model.bind_tools(tools)

question = "今日の日付を教えてください。。"

response = model_with_tools.invoke([HumanMessage(content=\
question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
tool = Get_date()
tool.invoke(response.tool_calls[0])

In [ ]:
tools = [Get_fortune(), Get_date()]

model = ChatOpenAI(model="gpt-4o-mini")
model_with_tools = model.bind_tools(tools)

question = "今日の運勢を教えてください。。"

response = model_with_tools.invoke([HumanMessage(content=question)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

## appendix

In [ ]:
# ツール定義（別パターン）

from langchain_core.tools import Tool

get_date_tool = Tool(
    name="Get_date",
    description ="今日の日付を取得する。インプットは 'date'です。'date' は、日付を取得する対象の日で、'今日', '明日', '明後日' という3種類の文字列から指定します。今日の日付を知りたい際は'今日'を入力します",
    func=get_date
)
get_fortune_tool = Tool(
    name="Get_fortune",
    description = "特定の日付の運勢を占う。インプットは  'date_string'です。'date_string' は、占いを行う日付で、mm月dd日 という形式です。1月1日の占いを行う際は'1月1日'を入力します",
    func=get_fortune
)

# 出力例
print(get_date_tool.invoke("今日"))
print(get_fortune_tool.invoke("10月23日"))